# Tester

In [1]:
import timm
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch.utils.data as data
import torchvision
from tqdm import tqdm, tqdm_notebook
import torch.nn.functional as F
import math

from vit_pooling import ViTPooling
from positional_enhancement import PositionalEnhanceViT, PositionalEnhanceViTv2


gpu_ids = []
device_names = []
if torch.cuda.is_available():
    for gpu_id in range(torch.cuda.device_count()):
        gpu_ids += [gpu_id]
        device_names += [torch.cuda.get_device_name(gpu_id)]
print(gpu_ids)
print(device_names)

if len(gpu_ids) > 1:
    gpu = 'cuda:' + str(gpu_ids[2])  # GPU Number
else:
    gpu = "cuda" if torch.cuda.is_available() else "cpu"

[0, 1, 2, 3]
['NVIDIA GeForce RTX 3090', 'NVIDIA GeForce RTX 3090', 'NVIDIA GeForce RTX 3090', 'NVIDIA GeForce RTX 3090']


In [2]:
device = gpu
model_path = './save/ViT_timm_vit_base_patch16_224_in21k_augPositive_i2012_ep8_lr0.00125.pt'
TIMM_MODEL = 'vit_base_patch16_224_in21k'
BATCH_SIZE = 512
NUM_EPOCHS = 8
NUM_WORKERS = 2
LEARNING_RATE = 1.25e-03

IMAGE_SIZE = 224
PATCH_SIZE = 16
IN_CHANNELS = 3
NUM_CLASSES = 1000
EMBED_DIM = 768
DEPTH = 12
NUM_HEADS = 12

In [3]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])
transform_test = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])
# pre_train_set = torchvision.datasets.ImageFolder('./data/ImageNet-21k', transform=transform_train)
# pre_train_loader = data.DataLoader(pre_train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
train_set = torchvision.datasets.ImageFolder('../../YJ/ILSVRC2012/train', transform=transform_train)
train_loader = data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_set = torchvision.datasets.ImageFolder('../../YJ/ILSVRC2012/val', transform=transform_test)
test_loader = data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

In [4]:
class TesterTimm(object):
    def __init__(self):
        self.model = None
        self.optimizer = None
        self.scheduler = None
        self.epochs = []
        self.losses = []

    def process(self):
        self.build_model()
        self.eval_model()

    def loss_checker(self):
        self.build_model()
        print(f'Steps: {len(self.epochs)}k steps')
        [print(f'Average Loss: {i:.3f}') for i in self.losses]
        
    def build_model(self):
        self.model = timm.create_model(TIMM_MODEL, pretrained=True).to(device)
        self.model.num_classes = NUM_CLASSES
        checkpoint = torch.load(model_path)
        self.epochs = checkpoint['epochs']
        self.model.load_state_dict(checkpoint['model'])
        self.losses = checkpoint['losses']
        print(f'Parameter: {sum(p.numel() for p in self.model.parameters() if p.requires_grad)}')
        print(f'Classes: {self.model.num_classes}')
        print(f'Epochs: {self.epochs[-1]}')
        
    def eval_model(self):
        self.model.eval()

        correct = 0
        total = 0
        with torch.no_grad():
            for i, data in tqdm_notebook(enumerate(test_loader, 0), total=len(test_loader)):
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                outputs = self.model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print(f'Accuracy of {len(test_set)} test images: {100 * correct / total:.2f} %')
        
    def lr_checker(self):
        self.build_model()
        model = self.model
        criterion = nn.CrossEntropyLoss()
        optimizer = SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)
        scheduler = CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)
        
        for epoch in range(NUM_EPOCHS):
            running_loss = 0.0
            saving_loss = 0.0
            print(optimizer.param_groups[0]['lr'])
            scheduler.step()

In [5]:
class TesterPaper(object):
    def __init__(self):
        self.model = None
        self.optimizer = None
        self.epochs = []
        self.losses = []
        self.cls_token = None

    def process(self):
        self.build_model()
        self.eval_model()
        
    def loss_checker(self):
        self.build_model()
        print(f'Steps: {len(self.epochs)}k steps')
        [print(f'Average Loss: {i:.3f}') for i in self.losses]

    def build_model(self):
        self.model = ViTPooling(image_size=IMAGE_SIZE,
                         patch_size=PATCH_SIZE,
                         in_channels=IN_CHANNELS,
                         num_classes=NUM_CLASSES,
                         embed_dim=EMBED_DIM,
                         depth=DEPTH,
                         num_heads=NUM_HEADS,
                         ).to(device)
        checkpoint = torch.load(model_path)
        self.epochs = checkpoint['epochs']
        self.model.load_state_dict(checkpoint['model'])
        self.losses = checkpoint['losses']
        print(f'Parameter: {sum(p.numel() for p in self.model.parameters() if p.requires_grad)}')
        print(f'Classes: {self.model.mlp_head.num_classes}')
        print(f'Epochs: {self.epochs[-1]}')

    def eval_model(self):
        self.model.eval()

        correct = 0
        total = 0
        with torch.no_grad():
            for i, data in tqdm_notebook(enumerate(test_loader, 0), total=len(test_loader)):
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                outputs = self.model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print(f'Accuracy of {len(test_set)} test images: {100 * correct / total:.2f} %')

In [6]:
class TesterPosition(object):
    def __init__(self):
        self.model = None
        self.optimizer = None
        self.scheduler = None
        self.epochs = []
        self.losses = []

    def process(self):
        self.build_model()
        self.eval_model()

    def loss_checker(self):
        self.build_model()
        print(f'Steps: {len(self.epochs)}k steps')
        [print(f'Average Loss: {i:.3f}') for i in self.losses]
        
    def build_model(self):
        self.model = PositionalEnhanceViT(NUM_CLASSES).to(device)
        self.model.num_classes = NUM_CLASSES
        checkpoint = torch.load(model_path)
        self.epochs = checkpoint['epochs']
        self.model.load_state_dict(checkpoint['model'])
        self.losses = checkpoint['losses']
        print(f'Parameter: {sum(p.numel() for p in self.model.parameters() if p.requires_grad)}')
        print(f'Classes: {self.model.num_classes}')
        print(f'Epochs: {self.epochs[-1]}')
        
    def eval_model(self):
        self.model.eval()

        correct = 0
        total = 0
        with torch.no_grad():
            for i, data in tqdm_notebook(enumerate(test_loader, 0), total=len(test_loader)):
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                outputs = self.model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print(f'Accuracy of {len(test_set)} test images: {100 * correct / total:.2f} %')
        
    def lr_checker(self):
        self.build_model()
        model = self.model
        criterion = nn.CrossEntropyLoss()
        optimizer = SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)
        scheduler = CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)
        
        for epoch in range(NUM_EPOCHS):
            running_loss = 0.0
            saving_loss = 0.0
            print(optimizer.param_groups[0]['lr'])
            scheduler.step()

In [7]:
if __name__ == '__main__':
    t = TesterTimm()
    t.loss_checker()
#     t.lr_checker()

Parameter: 102595923
Classes: 1000
Epochs: 8
Steps: 160k steps
Average Loss: 5.492
Average Loss: 2.038
Average Loss: 1.786
Average Loss: 1.694
Average Loss: 1.616
Average Loss: 1.549
Average Loss: 1.514
Average Loss: 1.461
Average Loss: 1.441
Average Loss: 1.408
Average Loss: 1.403
Average Loss: 1.368
Average Loss: 1.367
Average Loss: 1.341
Average Loss: 1.333
Average Loss: 1.321
Average Loss: 1.319
Average Loss: 1.292
Average Loss: 1.293
Average Loss: 1.271
Average Loss: 1.192
Average Loss: 1.171
Average Loss: 1.178
Average Loss: 1.167
Average Loss: 1.166
Average Loss: 1.159
Average Loss: 1.162
Average Loss: 1.144
Average Loss: 1.152
Average Loss: 1.133
Average Loss: 1.144
Average Loss: 1.148
Average Loss: 1.131
Average Loss: 1.131
Average Loss: 1.132
Average Loss: 1.126
Average Loss: 1.097
Average Loss: 1.114
Average Loss: 1.116
Average Loss: 1.104
Average Loss: 1.009
Average Loss: 0.997
Average Loss: 1.000
Average Loss: 1.004
Average Loss: 0.984
Average Loss: 0.998
Average Loss: 1.0

In [8]:
if __name__ == '__main__':
#     [t.process() for i in range(1)]

    for i in range(8):
        global model_path
        model_path = f'./save/ViT_timm_vit_base_patch16_224_in21k_augPositive_i2012_ep{i+1}_lr0.00125.pt'
        t.process()

Parameter: 102595923
Classes: 1000
Epochs: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/98 [00:00<?, ?it/s]

Accuracy of 50000 test images: 69.73 %
Parameter: 102595923
Classes: 1000
Epochs: 2


  0%|          | 0/98 [00:00<?, ?it/s]

...

Accuracy of 50000 test images: 76.34 %
Parameter: 102595923
Classes: 1000
Epochs: 7


  0%|          | 0/98 [00:00<?, ?it/s]

Accuracy of 50000 test images: 76.90 %
Parameter: 102595923
Classes: 1000
Epochs: 8


  0%|          | 0/98 [00:00<?, ?it/s]

Accuracy of 50000 test images: 77.38 %


In [9]:
# 프리 트레이닝
# 1스텝 = 9분
# 1에포크 = 3시간

# 파인 튜닝
# 1스텝 = 6분
# 1에포크 = 2시간